In [1]:
import pandas as pd
import transformers
from tqdm import tqdm
from transformers import pipeline

In [2]:
df = pd.read_csv('Data/complete_labeled_dataset.csv')
df.head()

,text,label
0,"Elon Musk is a piece of shit, greedy capitalis...",Toxic
1,The senile credit card shrill from Delaware ne...,Toxic
2,He does that a lot -- makes everyone look good...,Toxic
3,F*ck Lizzo,Toxic
4,Epstein and trump were best buds!!! Pedophiles...,Toxic


In [3]:
df = df.dropna()

In [4]:
sample = df.sample(n=200)

In [5]:
sample.head(4)

,text,label
29844,can we please stop talking about how some scu...,hate speech
11594,If the City Council and Mayor do whatever the ...,political
31312,//t.co/7oLD2eDB1R executed by Jihadis from #Tu...,hate speech
24925,When u fuck with crazy bitches shit like this...,offensive language


In [6]:
sample['label'].describe()

count           200
unique            6
top       political
freq             84
Name: label, dtype: object

In [29]:
df['label'].unique()

array(['Toxic', 'hate speech', 'sexual or profanity', 'spam', 'political',
       'offensive language'], dtype=object)

In [30]:
labels = ['toxic', 'racist', 'gender bias', 'religious bias', 'aggressive', 
          'personal attacks', 'hate speech', 'offensive language', 'sexual or profanity', 'spam', 'political']

In [31]:
dbert = pipeline("zero-shot-classification",
                      model="typeform/distilbert-base-uncased-mnli")

The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
All PyTorch model weights were used when initializing TFDistilBertForSequenceClassification.

All the weights of TFDistilBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecate

In [32]:
dbert_sentiment = []
dbert_score = []
for i in tqdm(sample.text, total=len(sample)):
    prediction = dbert(i, labels, multi_label=True)
    dbert_sentiment.append(prediction["labels"][0])
    dbert_score.append(prediction["scores"][0])

100%|██████████| 200/200 [07:45<00:00,  2.33s/it]


In [33]:
dbert_pred = pd.DataFrame([dbert_sentiment, dbert_score])
dbert_pred = dbert_pred.transpose()
dbert_pred = dbert_pred.rename(columns={0:'sentiment', 1:'score'} )
dbert_pred.head()

,sentiment,score
0,sexual or profanity,0.999815
1,political,0.774445
2,sexual or profanity,0.673865
3,sexual or profanity,0.999908
4,sexual or profanity,0.99953


In [34]:
dbert_pred.describe()

,sentiment,score
count,200,200.000000
unique,8,200.000000
top,sexual or profanity,0.999815
freq,148,1.000000


In [13]:
fbbart = pipeline("zero-shot-classification",
                      model="facebook/bart-large-mnli")

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBartForSequenceClassification: ['model.decoder.version', 'model.encoder.version']
- This IS expected if you are initializing TFBartForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBartForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBartForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBartForSequenceClassification for predictions without further training.


In [35]:
fbbart_sentiment = []
fbbart_score = []
for i in tqdm(sample.text, total=len(sample)):
    prediction = fbbart(i, labels, multi_label=True)
    fbbart_sentiment.append(prediction["labels"][0])
    fbbart_score.append(prediction["scores"][0])

100%|██████████| 200/200 [56:08<00:00, 16.84s/it] 


In [36]:
fbbart_pred = pd.DataFrame([fbbart_sentiment, fbbart_score])
fbbart_pred = fbbart_pred.transpose()
fbbart_pred = fbbart_pred.rename(columns={0:'sentiment', 1:'score'} )
fbbart_pred.head()

,sentiment,score
0,sexual or profanity,0.988856
1,toxic,0.982097
2,aggressive,0.971524
3,toxic,0.992957
4,sexual or profanity,0.839305


In [37]:
fbbart_pred.describe()

,sentiment,score
count,200,200.000000
unique,9,200.000000
top,sexual or profanity,0.988856
freq,51,1.000000


In [38]:
roberta = pipeline("zero-shot-classification", model="typeform/roberta-large-mnli")

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at typeform/roberta-large-mnli.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


In [39]:
roberta_sentiment = []
roberta_score = []
for i in tqdm(sample.text, total=len(sample)):
    prediction = roberta(i, labels, multi_label=True)
    roberta_sentiment.append(prediction["labels"][0])
    roberta_score.append(prediction["scores"][0])

100%|██████████| 200/200 [50:54<00:00, 15.27s/it]


In [40]:
roberta_pred = pd.DataFrame([roberta_sentiment, roberta_score])
roberta_pred = roberta_pred.transpose()
roberta_pred = roberta_pred.rename(columns={0:'sentiment', 1:'score'} )
roberta_pred.head()

,sentiment,score
0,toxic,0.986123
1,political,0.956578
2,aggressive,0.914582
3,personal attacks,0.99521
4,personal attacks,0.956599


In [41]:
roberta_pred.describe()

,sentiment,score
count,200,200.000000
unique,10,200.000000
top,political,0.986123
freq,65,1.000000


In [42]:
camembert = pipeline("zero-shot-classification", model="cmarkea/distilcamembert-base-nli")

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFCamembertForSequenceClassification: ['roberta.embeddings.position_ids']
- This IS expected if you are initializing TFCamembertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFCamembertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFCamembertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFCamembertForSequenceClassification for predictions without further training.


In [43]:
camembert_sentiment = []
camembert_score = []
for i in tqdm(sample.text, total=len(sample)):
    prediction = camembert(i, labels, multi_label=True)
    camembert_sentiment.append(prediction["labels"][0])
    camembert_score.append(prediction["scores"][0])

100%|██████████| 200/200 [09:18<00:00,  2.79s/it]


In [44]:
camembert_pred = pd.DataFrame([camembert_sentiment, camembert_score])
camembert_pred = camembert_pred.transpose()
camembert_pred = camembert_pred.rename(columns={0:'sentiment', 1:'score'} )
camembert_pred.head()

,sentiment,score
0,hate speech,0.830311
1,hate speech,0.748824
2,hate speech,0.909197
3,hate speech,0.963903
4,sexual or profanity,0.936252


In [45]:
camembert_pred.describe()

,sentiment,score
count,200,200.000000
unique,9,200.000000
top,hate speech,0.830311
freq,140,1.000000


Evaluating

In [ ]:
print(f"The average score or certainty for dbert was {dbert_pred['score'].mean()}")
print(f"The average score or certainty for fbbart was {fbbart_pred['score'].mean()}")
print(f"The average score or certainty for roberta was {roberta_pred['score'].mean()}")
print(f"The average score or certainty for camembert was {camembert_pred['score'].mean()}")

#### dbert has the shortest time per iteration followed by camembert at 1.73s/it and 1.94s/it respectively
#### camembert has the highest average score followed by roberta by some good distance at 84% and 76% respectively
#### camebert appears to be overfitting with 73% being hate speech
#### dbert overfits to sexual profanity
#### I am going to pick fbbart for its contextual accuracy over the rest

In [47]:
sample['label'].head()

29844           hate speech
11594             political
31312           hate speech
24925    offensive language
30379    offensive language
Name: label, dtype: object

In [50]:
dbert_pred['sentiment'].head()

0    sexual or profanity
1              political
2    sexual or profanity
3    sexual or profanity
4    sexual or profanity
Name: sentiment, dtype: object

In [51]:
fbbart_pred['sentiment'].head()

0    sexual or profanity
1                  toxic
2             aggressive
3                  toxic
4    sexual or profanity
Name: sentiment, dtype: object

In [55]:
roberta_pred['sentiment'].head()

0               toxic
1           political
2          aggressive
3    personal attacks
4    personal attacks
Name: sentiment, dtype: object

In [56]:
camembert_pred['sentiment'].head()

0            hate speech
1            hate speech
2            hate speech
3            hate speech
4    sexual or profanity
Name: sentiment, dtype: object

In [59]:
sample['text'][30379]

" @InfamousCoryOG @_WhitePonyJr_ @EakaErick there's only like one retarded fucking beaner that does and he's a stupid ass ex&#8230;"